##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Optimización de la ingeniería de aprendizaje automático con ML Metadata


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/mlmd/mlmd_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
  
</table>

Pensemos en un escenario en el que se configura una canalización de aprendizaje automático (ML) de producción para clasificar pingüinos. La canalización ingiere los datos de entrenamiento, entrena y evalúa un modelo y lo inserta en producción.

Sin embargo, luego, cuando intenta usar este modelo con un conjunto de datos más grande que contiene diferentes tipos de pingüinos, observa que su modelo no se comporta como se esperaba y comienza a clasificar las especies incorrectamente.

En este punto, tendrá las siguientes interrogantes:

- ¿Cuál es la forma más eficaz de depurar el modelo cuando el único artefacto disponible es el modelo en producción?
- ¿Qué conjunto de datos de entrenamiento se usó para entrenar el modelo?
- ¿Qué ejecución del entrenamiento condujo a este modelo erróneo?
- ¿Dónde están los resultados de la evaluación del modelo?
- ¿Por dónde empezar a depurar?

[ML Metadata (MLMD)](https://github.com/google/ml-metadata) es una biblioteca que aprovecha los metadatos asociados con los modelos de ML para ayudarlo a responder estas y otras preguntas. Una analogía útil es pensar en estos metadatos como el equivalente al inicio de sesión en el desarrollo de software. MLMD le permite hacer un seguimiento confiable de los artefactos y el linaje asociados con los diversos componentes de su canalización de ML.

En este tutorial, configurará una canalización de TFX para crear un modelo que clasifique a los pingüinos en tres especies según la masa corporal, la longitud y profundidad de sus cúlmenes, y la longitud de sus aletas. Luego, puede usar MLMD para hacer un seguimiento del linaje de los componentes de la canalización.

## Canalizaciones de TFX en Colab

Colab es un entorno de desarrollo sencillo que difiere significativamente de un entorno de producción. En producción, es posible que tenga varios componentes de canalización, como ingesta de datos, transformación, entrenamiento de modelos, historiales de ejecución, etc., en múltiples sistemas distribuidos. Para este tutorial, debe tener en cuenta que existen diferencias significativas en la orquestación y el almacenamiento de metadatos: todo se maneja localmente dentro de Colab. Si desea obtener más información sobre TFX en Colab, consulte [aquí](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras#background) .


## Preparación

En primer lugar, tenemos que instalar e importar los paquetes necesarios, configurar rutas y descargar datos.

### Actualización de pip

Para evitar actualizar Pip en un sistema cuando se ejecuta localmente, verifique que se esté ejecutando en Colab. Por supuesto, los sistemas locales se pueden actualizar por separado.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instalación e importación de TFX

In [ ]:
 !pip install -q tfx

### Importación de paquetes

#### ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe hacer clic en el botón "REINICIAR TIEMPO DE EJECUCIÓN" o usar el menú "Tiempo de ejecución &gt; Reiniciar tiempo de ejecución ..." para reiniciar el tiempo de ejecución. Esto se debe a la forma en que Colab carga los paquetes.

In [ ]:
import os
import tempfile
import urllib
import pandas as pd

import tensorflow_model_analysis as tfma
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

Verifique las versiones TFX y MLMD.

In [ ]:
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import ml_metadata as mlmd
print('MLMD version: {}'.format(mlmd.__version__))

## Cómo descargar el conjunto de datos

En esta instancia de Colab, usamos el [conjunto de datos Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) que está disponible en [Github](https://github.com/allisonhorst/palmerpenguins). Procesamos el conjunto de datos omitiendo los registros incompletos, eliminamos las columnas `island` y `sex`, y convertimos las etiquetas a `int32`. El conjunto de datos contiene 334 registros de la masa corporal, la longitud y profundidad de los cúlmenes de los pingüinos, y la longitud de sus aletas. Use estos datos para clasificar a los pingüinos en una de tres especies.

In [ ]:
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_root = tempfile.mkdtemp(prefix='tfx-data')
_data_filepath = os.path.join(_data_root, "penguins_processed.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

## Cómo crear un InteractiveContext

Para ejecutar componentes de TFX de forma interactiva en este bloc de notas, cree un `InteractiveContext`. `InteractiveContext` usa un directorio temporal con una instancia de base de datos de MLMD efímera. Tenga en cuenta que las llamadas a `InteractiveContext` no son operativas fuera del entorno de Colab.

En general, es una buena práctica agrupar ejecuciones de canalizaciones similares dentro de un `Context`.

In [ ]:
interactive_context = InteractiveContext()

## Cómo construir una canalización de TFX

Una canalización de TFX consta de varios componentes que procesan diferentes aspectos del flujo de trabajo de ML. En este bloc de notas, creará y ejecutará los componentes `ExampleGen`, `StatisticsGen`, `SchemaGen` y `Trainer` y usará los componentes `Evaluator` y `Pusher` para evaluar e insertar el modelo entrenado.

Consulte el [tutorial de componentes](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) para obtener más información sobre los componentes de la canalización de TFX.

Nota: La construcción de una canalización de TFX mediante la configuración de los componentes individuales implica una gran cantidad de código repetitivo. A los efectos de este tutorial, está bien si no comprende completamente cada línea de código en la configuración de la canalización. 

### Creación de una instancia y ejecución del componente ExampleGen

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
interactive_context.run(example_gen)

### Creación de una instancia y ejecución del componente StatisticsGen

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
interactive_context.run(statistics_gen)

### Creación de una instancia y ejecución del componente SchemaGen

In [ ]:
infer_schema = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
interactive_context.run(infer_schema)

### Creación de una instancia y ejecución del componente Trainer


In [ ]:
# Define the module file for the Trainer component
trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {trainer_module_file}

# Define the training algorithm for the Trainer module file
import os
from typing import List, Text

import tensorflow as tf
from tensorflow import keras

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

# Features used for classification - culmen length and depth, flipper length,
# body mass, and species.

_LABEL_KEY = 'species'

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema, batch_size: int) -> tf.data.Dataset:
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY), schema).repeat()


def _build_keras_model():
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  d = keras.layers.Dense(8, activation='relu')(d)
  d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])
  return model


def run_fn(fn_args: tfx.components.FnArgs):
  schema = schema_pb2.Schema()
  tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema)
  train_dataset = _input_fn(
      fn_args.train_files, fn_args.data_accessor, schema, batch_size=10)
  eval_dataset = _input_fn(
      fn_args.eval_files, fn_args.data_accessor, schema, batch_size=10)
  model = _build_keras_model()
  model.fit(
      train_dataset,
      epochs=int(fn_args.train_steps / 20),
      steps_per_epoch=20,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)
  model.save(fn_args.serving_model_dir, save_format='tf')

Ejecute el componente `Trainer`.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(trainer_module_file),
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=50))
interactive_context.run(trainer)

### Evaluación e inserción del modelo

Use el componente `Evaluator` para evaluar y "apruebe" el modelo antes de usar el componente `Pusher` para insertar el modelo en un directorio de servicio.

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(),
                                  'serving_model/penguins_classification')

In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='species', signature_name='serving_default')
    ],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='SparseCategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.6})))
        ])
    ],
    slicing_specs=[tfma.SlicingSpec()])

In [ ]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    schema=infer_schema.outputs['schema'],
    eval_config=eval_config)
interactive_context.run(evaluator)

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
interactive_context.run(pusher)

La ejecución de la canalización de TFX completa la base de datos MLMD. En la siguiente sección, se usará la API MLMD para consultar esta base de datos en busca de información de metadatos.

## Cómo consultar la base de datos MLMD

La base de datos MLMD almacena tres tipos de metadatos:

- Metadatos sobre la canalización e información de linaje asociada con los componentes de la canalización
- Metadatos sobre artefactos que se generaron durante la ejecución de la canalización
- Metadatos sobre las ejecuciones de la canalización

Una canalización típica de un entorno de producción sirve a múltiples modelos a medida que llegan nuevos datos. Cuando encuentre resultados erróneos en los modelos servidos, puede consultar la base de datos MLMD para aislar los modelos erróneos. Luego puede hacer un seguimiento del linaje de los componentes de la canalización que corresponden a estos modelos para depurar sus modelos.

Configure el almacén de metadatos (MD) con el `InteractiveContext` que se definió previamente para consultar la base de datos MLMD.

In [ ]:
connection_config = interactive_context.metadata_connection_config
store = mlmd.MetadataStore(connection_config)

# All TFX artifacts are stored in the base directory
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

Cree algunas funciones ayudantes para ver los datos del almacén MD.

In [ ]:
def display_types(types):
  # Helper function to render dataframes for the artifact and execution types
  table = {'id': [], 'name': []}
  for a_type in types:
    table['id'].append(a_type.id)
    table['name'].append(a_type.name)
  return pd.DataFrame(data=table)

In [ ]:
def display_artifacts(store, artifacts):
  # Helper function to render dataframes for the input artifacts
  table = {'artifact id': [], 'type': [], 'uri': []}
  for a in artifacts:
    table['artifact id'].append(a.id)
    artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
    table['type'].append(artifact_type.name)
    table['uri'].append(a.uri.replace(base_dir, './'))
  return pd.DataFrame(data=table)

In [ ]:
def display_properties(store, node):
  # Helper function to render dataframes for artifact and execution properties
  table = {'property': [], 'value': []}
  for k, v in node.properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  for k, v in node.custom_properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  return pd.DataFrame(data=table)

Primero, consulte el almacén de MD para obtener una lista de todos sus `ArtifactTypes` almacenados.

In [ ]:
display_types(store.get_artifact_types())

A continuación, consulte todos los artefactos `PushedModel`.

In [ ]:
pushed_models = store.get_artifacts_by_type("PushedModel")
display_artifacts(store, pushed_models)

Consulte el último modelo insertado en el almacén de MD. Este tutorial tiene un solo modelo insertado. 

In [ ]:
pushed_model = pushed_models[-1]
display_properties(store, pushed_model)

Uno de los primeros pasos para depurar un modelo insertado es observar qué modelo entrenado se inserta y qué datos de entrenamiento se usan para entrenar ese modelo.

MLMD ofrece API transversales para explorar el grafo de procedencia, que se puede usar para analizar la procedencia del modelo. 

In [ ]:
def get_one_hop_parent_artifacts(store, artifacts):
  # Get a list of artifacts within a 1-hop of the artifacts of interest
  artifact_ids = [artifact.id for artifact in artifacts]
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids(artifact_ids)
      if event.type == mlmd.proto.Event.OUTPUT)
  artifacts_ids = set(
      event.artifact_id
      for event in store.get_events_by_execution_ids(executions_ids)
      if event.type == mlmd.proto.Event.INPUT)
  return [artifact for artifact in store.get_artifacts_by_id(artifacts_ids)]

Consulte los artefactos principales para el modelo insertado.

In [ ]:
parent_artifacts = get_one_hop_parent_artifacts(store, [pushed_model])
display_artifacts(store, parent_artifacts)

Consulte las propiedades del modelo.

In [ ]:
exported_model = parent_artifacts[0]
display_properties(store, exported_model)

Consulte los artefactos ascendentes del modelo.

In [ ]:
model_parents = get_one_hop_parent_artifacts(store, [exported_model])
display_artifacts(store, model_parents)

Obtenga los datos de entrenamiento con los que entrenó el modelo.

In [ ]:
used_data = model_parents[0]
display_properties(store, used_data)

Ahora que tiene los datos de entrenamiento con los que se entrenó el modelo, consulte la base de datos nuevamente para encontrar el paso de entrenamiento (ejecución). Consulte el almacén de MD para obtener una lista de los tipos de ejecución registrados.

In [ ]:
display_types(store.get_execution_types())

El paso de entrenamiento es el `ExecutionType` denominado `tfx.components.trainer.component.Trainer`. Explore el almacén de MD para ejecutar el entrenador que corresponde al modelo insertado.

In [ ]:
def find_producer_execution(store, artifact):
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids([artifact.id])
      if event.type == mlmd.proto.Event.OUTPUT)
  return store.get_executions_by_id(executions_ids)[0]

trainer = find_producer_execution(store, exported_model)
display_properties(store, trainer)

## Resumen

En este tutorial, aprendimos cómo sacar provecho de MLMD para hacer un seguimiento del linaje de los componentes de su canalización de TFX y resolver problemas.

Para obtener más información sobre cómo usar MLMD, consulte estos recursos adicionales:

- [Documentación de la API de MLMD](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd)
- [Guía de MLMD](https://www.tensorflow.org/tfx/guide/mlmd)